In [400]:
import pandas as pd

df = pd.read_csv('food_list.csv',encoding='cp949')

data_to_insert = {'구분': '_', '음식명': '_'}
df = df.append(data_to_insert, ignore_index=True)

data = df[['구분', '음식명']] # 필요한 데이터만 가져오기

for i in range (len(data)): # ,가 포함되거나 한 글자인 카테고리명 변경 / 문자가 있거나 한 글자인 음식명 변경
	if (',' in data['구분'][i]):
		data.loc[i, "구분"] = data['구분'][i].replace(',', '_')

	if (len(data['구분'][i]) == 1):
		data.loc[i, "구분"] = data['구분'][i] + "_"

	if (len(data['음식명'][i]) == 1):
		data.loc[i, "음식명"] = data['음식명'][i] + "_"

	if (" / " in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace(" / ", '_')
	
	if ("/" in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace("/", '_')


/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_44814/3088158830.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data_to_insert, ignore_index=True)
/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_44814/3088158830.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i, "구분"] = data['구분'][i] + "_"
/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_44814/3088158830.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i, "구분"] = data['구분'][i].replace(',', '_')
/var/fold

In [401]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,2)) 
cv_category = cv.fit_transform(data['구분']) 
cv.vocabulary_ # 카테고리별 인덱스 번호 

cv.vocabulary_

{'면_': 20,
 '떡_': 16,
 '찜_': 49,
 '어류': 34,
 '구이': 7,
 '젓갈': 43,
 '조림': 44,
 '채소류': 50,
 '나물': 12,
 '선_': 28,
 '전_적': 40,
 '볶음': 25,
 '양념': 33,
 '회_': 58,
 '국_탕': 8,
 '과일류': 6,
 '서류': 27,
 '김치': 11,
 '밥_': 23,
 '죽_': 46,
 '한과': 56,
 '마른찬': 18,
 '갑각류': 1,
 '생채': 26,
 '음청류': 37,
 '고명': 3,
 '장아찌': 38,
 '육류': 36,
 '전골': 42,
 '청주': 51,
 '패류': 54,
 '해조류': 57,
 '만두': 19,
 '찌개': 48,
 '곡류': 4,
 '연체류': 35,
 '묵_': 22,
 '기타': 10,
 '쌈_': 32,
 '견과류': 2,
 '편육': 55,
 '두부': 15,
 '떡국': 17,
 '숙채': 30,
 '두류': 14,
 '곡류 견과류': 5,
 '증류주': 47,
 '버섯류': 24,
 '탕_': 53,
 '수제비': 29,
 '순대': 31,
 '나물 생채': 13,
 '전_적_튀김': 41,
 '초_': 52,
 '국수': 9,
 '족편': 45,
 '전_': 39,
 '무침': 21,
 '__': 0}

In [402]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_category = cosine_similarity(cv_category, cv_category)
sim_category = similarity_category.argsort()[:,::-1]
print(sim_category)
sim_category.shape

[[  0  80 709 ... 493 492 375]
 [559 725  26 ... 493 492   0]
 [405 675 503 ... 492 491   0]
 ...
 [559 725  26 ... 493 492   0]
 [733  32 154 ... 489 488   0]
 [751 246 255 ... 497 496   0]]


(752, 752)

In [403]:
cv = CountVectorizer(ngram_range=(1,2)) 
cv_menuname = cv.fit_transform(data['음식명'])
similarity_menuname = cosine_similarity(cv_menuname, cv_menuname)
sim_menuname = similarity_menuname.argsort()[:,::-1]
sim_menuname.shape

(752, 752)

In [404]:
cate_menu_co = (
	+ similarity_category * 0.8
	+ similarity_menuname * 0.2
)

cate_menu_co_idx = cate_menu_co.argsort()[:,::-1]

In [405]:
def remove_in_list(menu_list):
	for i in range (len(menu_list)):
		menu_list[i] = menu_list[i].replace('_', '')
	return menu_list

In [406]:
import numpy as np

def recommend_menu(df, sorted_idx, menu_name, top=15):
	lst = []
	
	# 유사도 계산해서 유사도순으로 추천
	for i in range (len(data)):
		lst.append(menu_name in df['음식명'][i])

	result = df.iloc[lst]
	return result['음식명'].values ## 추천 메뉴 전송 필요


In [419]:
menu_name = "돼지" ## 입력

lst = []

if (len(menu_name) == 1):
	menu_name += "_"

if (True in (data['음식명'] == menu_name).values): # 입력과 일치하는 메뉴가 있는 경우
		print("일치하는 메뉴가 있습니다.")
		lst.append(menu_name)
		
else : # 입력과 일치하는 메뉴명이 없는 경우
	lst = recommend_menu(data, cate_menu_co_idx, menu_name)
	if (len(lst) != 0):
		print("일치하는 메뉴가 없습니다. 추천 메뉴는 다음과 같습니다.")

if (len(lst) == 0):
	print("메뉴가 없습니다.")
else :
	lst = remove_in_list(lst)
	print(lst) ## 주문 메뉴 전송 필요

일치하는 메뉴가 없습니다. 추천 메뉴는 다음과 같습니다.
['돼지갈비강정' '돼지갈비구이' '돼지갈비찜' '돼지고기' '돼지고기구이' '돼지고기김치볶음' '돼지고기볶음' '돼지고기편육'
 '돼지머리편육']
